In [10]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



In [11]:
print(reddit.read_only)

True


In [12]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Hsb
--strikethru--
yet another test
This is a test title.
Chat's down
test
Sheep in Human Clothing
CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now
  AAAAA


In [13]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [14]:

stockCSV = pd.read_csv('stock_info.csv') #Read the CSV file

stockCSV['Ticker'] = '$' + stockCSV['Ticker'] #Add a $ sign in front of the ticker
stockCSV.head() #Display the first 5 rows of the dataframe


,Ticker,Name,Exchange
0,$A,Agilent Technologies,NYSE
1,$AA,Alcoa Inc.,NYSE
2,$AAN,Aaron's Inc,NYSE
3,$AAT,American Assets Trust,NYSE
4,$AAV,Advantage Oil & Gas Ltd,NYSE


In [ ]:

postData = []
import nltk  #Natural Language Toolkit
nltk.download('punkt') #sentence word tokenziation
import textstat #a simple python library to calculate statistics from text 
from textblob import TextBlob
from nltk.tokenize import TabTokenizer
from nltk.tokenize import word_tokenize
import datetime
buySell = 0

tokenizer = word_tokenize() #word tokenizer

# Fetch the top 1000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        #textWords = nltk.word_tokenize(submission.selftext) #tokenize the post text, pretty sure we do not need this
        wordsPost = submission.selftext.split() #split the title into words
        blob = TextBlob(submission.selftext, tokenizer=tokenizer) #create a textblob object
        print(blob.tokens) #print the sentiment polarity
        
        #determining buy sell depending on sentiment
        if(blob.sentiment.polarity >0):
            buySell = 1
        elif(blob.sentiment.polarity <0):
            buySell = -1
        else:
            buySell = 0
        
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif any(word in stockCSV['Ticker'].values for word in wordsPost): #check if any of the words in the post are stock abbreviations
            postData.append({
                 #These are indentifying data
                'stock abbreviation': next(word for word in wordsPost if word in stockCSV['Ticker'].values),
                'stock name': stockCSV.loc[stockCSV['Ticker'] == next(word for word in wordsPost if word in stockCSV['Ticker'].values)]['Name'].values[0],
                'title': submission.title,
                'date': datetime.datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': submission.id,
                'url': submission.url, 
                'post text': submission.selftext, #post text
                #All of the following are features that can be used to predict the stock price
                'upvote Ratio': submission.upvote_ratio, #upvote ratio
                'comment number': submission.num_comments, #number of comments
                'text length': len(submission.selftext),#length of the post text
                'flesh_kincaid_grade': textstat.flesch_kincaid_grade(submission.selftext), #Flesch Kincaid Grade
                'lexical_diversity': len(set(wordsPost)) / len(wordsPost), #lexical diversity
                'Sentiment (polarity):': blob.sentiment.polarity, #Sentiment
                'Sentiment (subjectivity)': blob.sentiment.subjectivity, #Subjectivity
                'Buy/Sell': buySell #Buy or Sell
            })
        else:
            continue
        

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: word_tokenize() missing 1 required positional argument: 'text'

In [ ]:
df = pd.DataFrame(postData)
df 

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesh_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell
0,$MSTR,Microstrategy Inc.,100% $MSTR on Roth IRA,2024-11-20 13:23:59,1gvwat8,https://i.redd.it/hauhtuozm32e1.jpeg,Gains on Microstrategy $MSTR will be tax free ...,0.85,6,204,4.7,0.868421,0.225000,0.625000,1
1,$CRM,Salesforce.Com Inc,How to invest going into 2025 according to me,2024-11-20 19:34:36,1gw35du,https://www.reddit.com/r/wallstreetbets/commen...,Hello fellow regards. Many of you have been as...,0.92,13,2369,12.2,0.638462,0.103828,0.410705,1
2,$MSTR,Microstrategy Inc.,YOLO $MSTR... Shorting this thing will get you...,2024-11-19 13:15:25,1gv3unh,https://www.reddit.com/r/wallstreetbets/commen...,https://preview.redd.it/220iqucsfw1e1.png?widt...,0.92,431,419,5.8,0.877193,-0.100000,0.193333,-1
3,$NVDA,Nvidia Corp.,Let it $SNOW,2024-11-19 15:34:43,1gv77n8,https://www.reddit.com/r/wallstreetbets/commen...,"Let's face it, $SNOW has been one of the most ...",0.92,264,2821,8.0,0.660907,0.127396,0.451457,1
4,$MSTR,Microstrategy Inc.,$MSTR is real. In since Oct 28th,2024-11-19 22:46:26,1gvgl5i,https://www.reddit.com/gallery/1gvgl5i,"Started with 50k. I just bout and held $MSTU, ...",0.83,33,141,2.1,0.875000,-0.025000,0.325000,-1
5,$NVDA,Nvidia Corp.,$VRT ➡️ S&P 500: Coming Soon,2024-11-19 14:49:34,1gv6504,https://www.reddit.com/r/wallstreetbets/commen...,Vertiv ($VRT) has been ripping all year and is...,0.92,66,5630,11.8,0.653113,0.039537,0.455942,1
6,$SAVE,Spirit Airlines Inc.,What happens to $SAVE ITM puts,2024-11-18 21:17:07,1gumhfc,https://www.reddit.com/r/wallstreetbets/commen...,"After filing for Chapter 11 bankruptcy, share...",0.77,23,413,3.3,0.810811,-0.200000,0.491667,-1
7,$KO,Coca-Cola Company,Food And Healthcare Weekly/Monthly Calls Play,2024-11-18 10:15:35,1gu6zu3,https://i.redd.it/yltadvzjfo1e1.jpeg,"In my previous post, I was talking about my st...",0.90,11,352,4.0,0.878788,-0.006667,0.446667,-1
8,$TSLA,Tesla Motors,$TSLA $400 calls exp Dec 6th - am I an idiot?,2024-11-17 12:00:33,1gti1cw,https://www.reddit.com/r/wallstreetbets/commen...,During the Trump rally (Nov 11th) I bought $TS...,0.92,519,124,4.2,0.958333,0.000000,0.000000,0
9,$DIS,Walt Disney Company,$OKLO shares and options YOLO,2024-11-17 18:29:38,1gtquma,https://www.reddit.com/r/wallstreetbets/commen...,https://preview.redd.it/nscyyvripj1e1.png?widt...,0.88,43,1085,13.6,0.754386,0.054167,0.508333,1


In [ ]:
#TO DO 
#1. Add Data from Yahoo Finance API to the dataframe
#2. Get the "buy/sell" value of that stock for the day
#3. See what happended a day later
#4. Get the Delta between the two values
#5. Use that to create a model that predicts the stock price
from nltk.tokenize import word_tokenize
newBlob = TextBlob("Je ne sais pas comment je vais reussir mais je devrai pour mon pays ma mere ma pere pour mon dieu pour mon copine - Napolean 1805", tokenizer=word_tokenize)
newBlob.tokens

ValueError: tokenizer must be an instance of BaseTokenizer